In [1]:
import pandas as pd
import numpy as np
import pyodbc
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import sqlite3
from sqlalchemy import create_engine

In [ ]:
conn = create_engine(r'mssql+pyodbc://@PYTHONSERVER\SQLEXPRESS/InSync?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes', fast_executemany=True)

In [ ]:
import sys
import logging
path = r'..\..\Logs\clinical_log.log'
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger('TreatmentPlanReport')
# logger.info('testing log')

## Notes
- Make sure that due dates are not calculated by time
- Do we want to see which plans were signed/closed late???
- If there are two initial treatment plans go by the second one.
<br>

## To-Do

In [ ]:
try:
    logger.info(f'Querying ptPatient_Activity.')
    sql='''
    SELECT 
        ptPatient_Activity.*,
        MRNNumber,
        LastName + ', ' + FirstName as patient_fullName,
        DOB,
        FacilityName AS 'Primary Facility'
    FROM 
        ptPatient_Activity
        LEFT JOIN emr_PatientDetails ON (ptPatient_Activity.PatientID=emr_PatientDetails.PatientID)
        LEFT JOIN emr_FacilitiesDetails ON (emr_PatientDetails._PrimaryFacility = emr_FacilitiesDetails.FacilityId)
    '''
    patientDetails = pd.read_sql(sql, conn)
    patientDetails = patientDetails.replace('', np.nan)
    patientDetails['DOB'] =pd.to_datetime(patientDetails['DOB'])
    logger.info(f'Successfully queried ptPatient_Activity.')
except Exception as e:
    print(e)
    logger.error(f'Failed to query ptPatient_Activity.', exc_info=True) 

## Get Treatment Plan Info

In [ ]:
try:
    logger.info(f'Querying treatment plan related info from tblEncounterSummary.')
    sql='''
    SELECT 
        tblEncounterSummary.PatientId as PatientID,
        tblEncounterSummary.EncounterID, 
        tblResourceDetails.FullName as ProviderName,
        tblEncounter.EndDate,
        VisitDateTime as EncounterDate,
        EncounterEndDateTimeToolTip,
        EncounterStartDateTimeToolTip,
        FacilityName,
        tblEncounterType.EncounterType,
        EncounterOpenClosed,
        ProgramName,
        closedByLastName + ', ' + closedByFirstName AS closedByFullName
    FROM 
        tblEncounterSummary
        LEFT JOIN emr_FacilitiesDetails ON (tblEncounterSummary.FacilityId = emr_FacilitiesDetails.FacilityId)
        LEFT JOIN tblEncounterEndDetails ON (tblEncounterSummary.EncounterID = tblEncounterEndDetails.EncounterID)
        INNER JOIN tblEncounter ON (tblEncounterSummary.EncounterID = tblEncounter.EncounterID)
        LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID = tblEncounterType.EncounterTypeID)
        LEFT JOIN tblResourceDetails ON (tblEncounterSummary.ProviderId = tblResourceDetails.ResourceId)
    WHERE (tblEncounterType.EncounterType LIKE '%treatment%'
        OR tblEncounterType.EncounterType LIKE '%transfer%'
        OR tblEncounterType.EncounterType LIKE '%Initial Eval%'
        OR tblEncounterType.EncounterType LIKE '%discharge%')
        AND tblEncounterType.EncounterType NOT LIKE '%CFTSS%'
    '''
    treatmentPlanEncounters = pd.read_sql(sql, conn)
    # replaces empty strings in the treatmentPlanEncounters DataFrame with NaN values to handle missing data appropriately
    treatmentPlanEncounters = treatmentPlanEncounters.replace('', np.nan)
    treatmentPlanEncounters['EncounterDate'] = pd.to_datetime(treatmentPlanEncounters['EncounterDate'])
    treatmentPlanEncounters['EndDate'] = pd.to_datetime(treatmentPlanEncounters['EndDate'])
    # Remove the word time: from column
    treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = treatmentPlanEncounters['EncounterStartDateTimeToolTip'].str.replace('Time: ', '')
    treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = treatmentPlanEncounters['EncounterEndDateTimeToolTip'].str.replace('Time: ', '')

    treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterStartDateTimeToolTip'])
    treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterEndDateTimeToolTip'])

    logger.info(f'Successfully queried treatment plan related info from tblEncounterSummary.')
except Exception as e:
    print(e)
    logger.error(f'Failed to query treatment plan related info from tblEncounterSummary.', exc_info=True) 

## Check Patients for Admission

In [ ]:
try:
    sql = f'''
    SELECT
    PatientId as PatientID,
    VisitDateTime,
    tblEncounterType.Encountertype
    FROM 
        tblEncounterSummary
    LEFT JOIN
        tblEncounterType ON tblEncounterType.EncounterTypeID = tblEncounterSummary.EncounterTypeID
    WHERE 
        tblEncounterType.EncounterType LIKE '%Admission Note%' 
        AND tblEncounterType.EncounterType NOT LIKE '%CFTSS%' 
    '''
    missingAdmissionNote = pd.read_sql(sql,conn)
    missingAdmissionNote['VisitDateTime'] = pd.to_datetime(missingAdmissionNote['VisitDateTime'])
    missingAdmissionNote = missingAdmissionNote.sort_values(by=['PatientID', 'VisitDateTime'], ascending=False)
    missingAdmissionNote = missingAdmissionNote.drop_duplicates('PatientID')
    missingAdmissionNote['HasAdmissionNote'] = True
    logger.info(f'Successfully queried admitted patients. Patient must have an admission note.')
except Exception as e:
    print(e)
    logger.error(f'Failed to query admitted patients. Patient must have an admission note.', exc_info=True) 

In [ ]:
try:
    sql = f'''
    SELECT
    PatientId as PatientID,
    tblEncounterType.EncounterType
    FROM 
        tblEncounterSummary
    LEFT JOIN
        tblEncounterType ON tblEncounterType.EncounterTypeID = tblEncounterSummary.EncounterTypeID
    WHERE 
        tblEncounterType.EncounterType LIKE '%Psychotherapy%' 
        OR 
        (tblEncounterType.EncounterType LIKE '%Initial Eval%' 
        AND tblEncounterType.EncounterType LIKE '%2%')
        OR
        tblEncounterType.EncounterType = 'Admission Note' 
    '''
    relevantEncounters = pd.read_sql(sql,conn)
    # groupby patient IDs and then check encounter list to see if either they have both an initial evaluation and a psychotherapy or an admission note.
    acceptedPatients = relevantEncounters.groupby(['PatientID'])['EncounterType'].apply(lambda x: (( True in list(x.str.lower().str.contains('psychotherapy')) ) 
                                                                                & ( True in list(x.str.lower().str.contains('initial eval'))  )  )
                                                                               | (True in list(x.str.lower() == 'admission note'))
                                                                       )
    acceptedPatients = acceptedPatients.reset_index()        
    acceptedPatients.columns = ['PatientID', 'Admitted']    
    treatmentPlanEncounters= treatmentPlanEncounters.merge(acceptedPatients, on='PatientID', how='left', validate='m:1')
    treatmentPlanEncounters['Admitted'].fillna('False', inplace=True)
    logger.info(f'Successfully queried admitted patients. Patient must have either an encounter type of psychotherapy or an initial eval 2.')
except Exception as e:
    print(e)
    logger.error(f'Failed to query admitted patients. Patient must have either an encounter type of psychotherapy or an initial eval 2.', exc_info=True) 

## Check for Primary Providers

In [ ]:
try:
    sql = f'''
    SELECT
    PatientId as PatientID,
    tblResourceDetails.FullName as Primary_Provider
    FROM 
        tblPatientPrimaryProviders
    LEFT JOIN
        tblResourceDetails ON (tblPatientPrimaryProviders.PatientPrimaryProviderId = tblResourceDetails.ResourceId)
    WHERE 
        IsDefault = 'TRUE'
    '''
    primaryProvider = pd.read_sql(sql,conn)
    primaryProvider.drop_duplicates('PatientID', inplace=True)
    treatmentPlanEncounters= treatmentPlanEncounters.merge(primaryProvider, on='PatientID', how='left', validate='m:1')
    treatmentPlanEncounters['Primary_Provider'].fillna('Unknown', inplace=True)
    logger.info(f'Successfully queried primary providers.')
except Exception as e:
    print(e)
    logger.error(f'Failed to query primary providers.', exc_info=True) 

## Treatment Plan Functions

In [ ]:
def find_missing_initial_treatment_plan(temp_df):
    # if patient does not have initial treatment plan 
    # and first treatment plan review date is more than a year past our InSync onboarding 
    # and patient was admitted, 
    # return true otherwise return false
    if 'Initial Treatment Plan' not in temp_df['EncounterType'].unique() and str(temp_df[temp_df['Summarized Encounter Type'] == 'Treatment Plan']['EncounterDate'].first_valid_index()) != 'None' and str(temp_df[temp_df['Admitted'] == True]['Admitted'].first_valid_index()) != 'None':
        if temp_df.loc[temp_df[temp_df['Summarized Encounter Type'] == 'Treatment Plan']['EncounterDate'].first_valid_index(), 'EncounterDate'] > pd.Timestamp('7/20/2022'):
            return True
        else:
            return False
    else:
        return False
    
def getDischargeInfo(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return '''Last encounter was discharge so Patient should probably be inactivated | '''    

def filterAndSortToRelevantData(completeDataFrame, patientID):
    singlePatient_df = completeDataFrame[completeDataFrame['PatientID'] == patientID].copy()
    singlePatient_df = singlePatient_df[singlePatient_df['Summarized Encounter Type'].map(lambda encType: True if encType == 'Treatment Plan' or encType == 'Transfer Note' or encType == 'Discharge Note' else False)].copy()
    singlePatient_df = singlePatient_df.sort_values(by='Review Date', ascending=True)
    singlePatient_df = singlePatient_df.reset_index(drop=True)
    return singlePatient_df


def isEmptyDataFrame(singlePatient_df):
    if singlePatient_df.shape[0] == 0:
        return True
    else:
        return False
    
def isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return True  
    else:
        return False

def isTransferNote(singlePatient_df, currentEncounterIndexNum):
    if singlePatient_df[currentEncounterIndexNum, 'Summarized Encounter Type'] == 'Transfer Note':
        return True
    else:
        return False

def hasNextPlan(singlePatient_df, currentEncounterIndexNum):
    # and see if there are any treatment plans after current treatment plan
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if 'Treatment Plan' in singlePatient_df[singlePatient_df['EncounterDate'] > currentTreatmentPlanDate]['Summarized Encounter Type'].unique():
        return True
    else:
        return False

def getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    nextPlanLocation = singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & (singlePatient_df['Summarized Encounter Type'] == 'Treatment Plan') ].index[0]
    return nextPlanLocation

def isTransferedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Transfer Note' in singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]['Summarized Encounter Type'].unique():
            return True
        else:
            return False
    else:
        if 'Transfer Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]['Summarized Encounter Type'].unique():
            return True
        else:
            return False
    
def isDischargedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan: # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Discharge Note' in singlePatient_df[(singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate) & 
                                           ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]['Summarized Encounter Type'].unique():
            return True
        else:
            return False
    else: # see if there is a 
        if 'Discharge Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]['Summarized Encounter Type'].unique():
            return True
        else:
            return False    
    
def isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation):
    if singlePatient_df.loc[nextPlanLocation, 'EndDate']!= pd.NaT and singlePatient_df.loc[nextPlanLocation, 'EndDate'] > singlePatient_df.loc[currentEncounterIndexNum, 'Review Date']:
        return True
    else:
        return False

def isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum):
    if str(singlePatient_df.loc[currentEncounterIndexNum, 'closedByFullName']) =='None':
        return False
    else:
        return True

def getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        return (nextTreatmentPlanDate-singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']).days
    else:
        daysAfterTransfer = (datetime.now()-currentTreatmentPlanDate).days
        return daysAfterTransfer



In [ ]:
try:
    relevantInfodf = treatmentPlanEncounters.merge(patientDetails, how='left', on='PatientID', validate='m:1')
    relevantInfodf = relevantInfodf[relevantInfodf['MRNNumber'].notnull()].copy()
    logger.info(f'Successfully merged patient details with treatment plan info.')
except Exception as e:
    print(e)
    logger.error(f'Failed to merge patient details with treatment plan info.', exc_info=True) 

## Calculate Review Date

In [ ]:
try:
    relevantInfodf['Summarized Encounter Type'] = relevantInfodf['EncounterType'].map(lambda type: 
                                                              'Treatment Plan' if 'treatment' in type.lower() else 
                                                               ('Initial Eval' if 'evaluation' in type.lower() else 
                                                                ('Transfer Note' if 'transfer' in type.lower() else 
                                                                 ('Discharge Note' if 'discharge' in type.lower() else 'Something else'))))
    # Cut off date for switching to 6 month treatment plans is September 23rd 2023 all previously made treatment plans follow old rules
    relevantInfodf['Review Date'] = relevantInfodf.apply(lambda row: 
                                                         row['EncounterDate'] + relativedelta(days=+30) if 'Initial Eval' in str(row['EncounterType'] )
                                                         else (row['EncounterDate'] + relativedelta(months=+6) if row['EncounterDate'] > datetime.strptime('09/23/23 00:00:00', '%m/%d/%y %H:%M:%S')
                                                         else (row['EncounterDate'] + relativedelta(months=11) if row['EncounterType'] == 'Initial Treatment Plan' else  
                                                         row['EncounterDate'] + relativedelta(months=+12))), axis=1)
    relevantInfodf['Days to Review Date'] = relevantInfodf['Review Date'].map(lambda reviewDate:  reviewDate.date()) - date.today()
    logger.info(f'Successfully calculated review date.')
except Exception as e:
    print(e)
    logger.error(f'Failed to calculate review date.', exc_info=True) 

## Determine Treatment Plan Status

In [ ]:
missingInitialTreatmentPlan = f'Missing Initial Treatment Plan'

try:
    allPatientDf = pd.DataFrame()
    for patientID in relevantInfodf['PatientID'].unique()[0:]:
        singlePatient_df = filterAndSortToRelevantData(completeDataFrame=relevantInfodf, patientID=patientID)
        if isEmptyDataFrame(singlePatient_df): # if patient has initial eval with no initial treatment plan
            emptySinglePatient_df = relevantInfodf[(relevantInfodf['PatientID'] == patientID)].copy()

            emptySinglePatient_df['missingInitialTreatmentPlan'] = find_missing_initial_treatment_plan(emptySinglePatient_df)
            emptySinglePatient_df['treatmentPlanStatus'] = missingInitialTreatmentPlan

            allPatientDf = pd.concat([allPatientDf,emptySinglePatient_df])
            continue
        else:
            patientOverallStatus = ''
            
            # check if patient is missing 
            is_missingInitialTreatmentPlan = find_missing_initial_treatment_plan(singlePatient_df)
            singlePatient_df['missingInitialTreatmentPlan'] = is_missingInitialTreatmentPlan   
            patientOverallStatus += missingInitialTreatmentPlan
            singlePatient_df['treatmentPlanStatus'] = missingInitialTreatmentPlan
            if is_missingInitialTreatmentPlan:
                patientOverallStatus = 'Missing Initial Treatment Plan'

            is_ActiveInInSync = singlePatient_df.loc[0, 'is_ActiveInInSync']
            is_lastEncounterDischarge = singlePatient_df.loc[0, 'is_lastEncounterDischarge']

            is_NeedToBeInactivated = isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge)
            singlePatient_df['is_NeedToBeInactivated'] = is_NeedToBeInactivated

            for currentEncounterIndexNum in range(singlePatient_df.shape[0]):
                treatmentPlanStatus = patientOverallStatus
                currentEncounterType = singlePatient_df.loc[currentEncounterIndexNum, 'Summarized Encounter Type']
                daysToReviewDate = singlePatient_df.loc[currentEncounterIndexNum, 'Days to Review Date'].days

                has_NextPlan = hasNextPlan(singlePatient_df, currentEncounterIndexNum)
                singlePatient_df.loc[currentEncounterIndexNum, 'has_NextPlan'] = has_NextPlan

                is_CurrentPlanSigned = isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum)
                singlePatient_df.loc[currentEncounterIndexNum, 'is_CurrentPlanSigned'] = is_CurrentPlanSigned

                if not is_CurrentPlanSigned:
                    treatmentPlanStatus += 'Plan Still needs to be signed | '       

                if is_NeedToBeInactivated:
                    treatmentPlanStatus += 'Last encounter was discharge so Patient should probably be inactivated | '
                if has_NextPlan:
                    next_PlanLocation = getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum)
                    singlePatient_df.loc[currentEncounterIndexNum, 'next_PlanLocation'] = next_PlanLocation

                    is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, 'is_TransferedNext'] = is_TransferedNext

                    is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, 'is_DischargedNext'] = is_DischargedNext
                    if (is_TransferedNext and not is_DischargedNext) and is_CurrentPlanSigned and has_NextPlan:
                        get_DaysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                        treatmentPlanStatus += f'Old Plan is Good | Patient was Transfered and they completed the follow up plan {get_DaysAfterTransfer} days after.'
                    elif (is_DischargedNext and not is_TransferedNext) and is_CurrentPlanSigned and has_NextPlan:
                        treatmentPlanStatus += f'Patient was Discharged'
                    elif not is_TransferedNext and not is_DischargedNext and is_CurrentPlanSigned and has_NextPlan:
                        treatmentPlanStatus += 'Everything Seems Good'

                    is_NextPlanClosedLate = isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    singlePatient_df.loc[currentEncounterIndexNum, 'is_NextPlanClosedLate'] = is_NextPlanClosedLate

                else: # if there is no next plan 
                    is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                    singlePatient_df.loc[currentEncounterIndexNum, 'is_TransferedNext'] = is_TransferedNext

                    is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                    singlePatient_df.loc[currentEncounterIndexNum, 'is_DischargedNext'] = is_DischargedNext

                    if not is_TransferedNext and not is_DischargedNext and daysToReviewDate > 0:
                        treatmentPlanStatus += f'Everything Seems Good. Still have {daysToReviewDate} days left.'
                    elif not is_TransferedNext and not is_DischargedNext and daysToReviewDate < 0:
                        treatmentPlanStatus += f'Plan is {np.absolute(daysToReviewDate)} days overdue.'
                    elif is_TransferedNext and not is_DischargedNext:
                        daysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                        if daysAfterTransfer > 30 and currentEncounterType == 'Transfer Note':
                            treatmentPlanStatus += f'Patient Transfered. Plan is {daysAfterTransfer-30} days overdue.'
                        elif daysAfterTransfer > 30:
                            treatmentPlanStatus += f'Patient Transfered.'
                        else:
                            treatmentPlanStatus += f'Patient Transfered. Plan is due in {30 - daysAfterTransfer} days.'
                    elif (is_DischargedNext and not is_TransferedNext):
                        treatmentPlanStatus += f'Patient was Discharged'   
                singlePatient_df.loc[currentEncounterIndexNum, 'treatmentPlanStatus'] = treatmentPlanStatus  
            allPatientDf = pd.concat([allPatientDf,singlePatient_df])
    logger.info(f'Successfully built all patients treatment plan details.')
except Exception as e:
    print(e)
    logger.error(f'Failed to build all patients treatment plan details.', exc_info=True) 

In [ ]:
allPatientDf[['has_NextPlan', 
              'is_CurrentPlanSigned',
              'next_PlanLocation',
              'is_TransferedNext',
              'is_DischargedNext',
              'is_NextPlanClosedLate']] = allPatientDf[['has_NextPlan', 
                                                        'is_CurrentPlanSigned',
                                                        'next_PlanLocation',
                                                        'is_TransferedNext',
                                                        'is_DischargedNext',
                                                        'is_NextPlanClosedLate']].fillna(False)   

In [ ]:
# for column in allPatientDf.columns:
#     print(column)

In [ ]:
try:
    allPatientDf['is_TreatmentPlanGood'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'Everything Seems Good' in str(status) else False)
    allPatientDf['is_TreatmentPlanOverdue'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'overdue' in str(status).lower() else False)
    logger.info(f'Successfully marked treatment plans that are all good.')
except Exception as e:
    print(e)
    logger.error(f'Failed to mark treatment plans that are all good.', exc_info=True) 

In [ ]:
allPatientDf['treatmentPlanStatus'] = allPatientDf['treatmentPlanStatus'] + '| ' + allPatientDf['seen_Recently']

In [ ]:
# final_df[final_df['PatientID']==622214]
# final_df[final_df['PatientID']==622741]

In [ ]:
allPatientDf['seen_Recently'].unique()

In [ ]:
# final_df[final_df['allTheInfoYouEverWanted'].map(lambda x: True if 'Closed After Due Date' in x else False) ].groupby('ProviderName')['EncounterId'].count().sort_values(ascending=False).head(20)

In [ ]:
# final_df[final_df['allTheInfoYouEverWanted']=='']

In [ ]:
# Change to integer so SQL can recognize it
# allPatientDf['Days to Review Date'] = pd.to_datetime(allPatientDf['Days to Review Date'])
allPatientDf['Days to Review Date'] = allPatientDf['Days to Review Date'].map(lambda timedelta: timedelta.days)

In [ ]:
allPatientDf = allPatientDf.merge(missingAdmissionNote[['PatientID', 'HasAdmissionNote']], how='left', on='PatientID')


In [ ]:
allPatientDf['HasAdmissionNote'] = allPatientDf['HasAdmissionNote'].fillna(False)

In [ ]:
allPatientDf.columns

In [ ]:
# filter to useful columns
allPatientDf = allPatientDf[['EncounterID',
                            'MRNNumber',
                            'DOB',
                            'PatientID',
                            'ProviderName',
                            'Primary_Provider',
                            'patient_fullName',
                            'FacilityName',
                            'Summarized Encounter Type',
                            'EncounterDate',
                            'EncounterEndDateTimeToolTip',
                            'EncounterStartDateTimeToolTip',
                            'EncounterType',
                            'EncounterOpenClosed',
                            'ProgramName',
                            'closedByFullName',
                            'finalEncounterType',
                            'finalEncounterDate',
                            'is_ActiveInInSync',
                            'is_lastEncounterDischarge',
                            'lastBillable_EncounterType',
                            'lastBillable_EncounterDate',
                            'lastBillable_ProgramName',
                            'lastBillable_FacilityName',
                            'seen_Recently',
                            'Review Date',
                            'Days to Review Date',
                            'is_NeedToBeInactivated',
                            'has_NextPlan',
                            'is_CurrentPlanSigned',
                            'next_PlanLocation',
                            'is_TransferedNext',
                            'is_DischargedNext',
                            'is_NextPlanClosedLate',
                            'treatmentPlanStatus',
                            'missingInitialTreatmentPlan',
                            'HasAdmissionNote',
                            'is_TreatmentPlanGood',
                            'is_TreatmentPlanOverdue',
                            'Admitted']]

In [ ]:
allPatientDf = allPatientDf.drop_duplicates('EncounterID', keep='first')

## For Presenting
- Replace all names with random ones for presenting

In [ ]:
# randomNames = pd.read_excel('../data/Random Names.xlsx')

# final_df['patient_fullName'] = final_df['patient_fullName'].map(lambda name: list(randomNames['Names'].sample(1))[0])
# final_df['ProviderName'] = final_df['ProviderName'].map(lambda name: list(final_df['ProviderName'].sample(1))[0])

In [ ]:
rename_mapping = {
    'Summarized Encounter Type': 'Summarized_Encounter_Type',
    'Review Date': 'Review_Date',
    'Days to Review Date': 'Days_to_Review_Date',
}
# Rename columns using the mapping dictionary
allPatientDf = allPatientDf.rename(columns=rename_mapping)

In [ ]:
table_name= 'ptTreatmentPlanDuenessSummary'
try:
    allPatientDf.to_sql('ptTreatmentPlanDuenessSummary', conn, if_exists='replace', index = False)
    logger.info(f'Successfully pushed ptTreatmentPlanDuenessSummary to database.')
except Exception as e:
    logger.error(f'Failed to push ptTreatmentPlanDuenessSummary to database.', exc_info=True) 
    print(e)

In [ ]:
conn.dispose()